## Classifying texts with machine learning

In this notebook you can use the SVM (Support Vector Machine) algorithm, through the SGDClassifier estimator, already tested in the original database.

With this notebook it is possible to use the algorithm with the best performance to fit the nature of the data and obtain the ratings of the Dislike and Complain type evaluations for a new data set.

## Loading used libraries

In [1]:
#Bibliotecas para Pré-Processamento
import pandas as pd
pd.set_option('max_colwidth', 300)
import numpy as np
#!pip install texthero
#import texthero as hero
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

#Bibliotecas para Modelagem
# um classificador linear que utiliza o Gradiente Descendente Estocástico como método de treino. #Por padrão, utiliza o estimador SVM.
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

# Database reading

In [2]:
df = pd.read_csv('Base_Produtos.csv', sep= ';', encoding="latin-1")
df.shape

C:\Users\Windows10\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (0,11,34,35,36,37) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(26427, 43)

In [3]:
df.dropna(subset=['Verb Native'], inplace=True)
df.shape

(17128, 43)

In [6]:
df1=df.loc[df['TIPO']!='#N/D']
df1.dropna(subset=['TIPO'], inplace=True)
df1.shape

C:\Users\Windows10\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


(17064, 43)

In [7]:
#Seleção das avaliações acima de 4 caracteres
df1 = df1[df1['Verb Native'].map(len) > 4]
df1.shape

(17009, 43)

## Separating the columns

In [8]:
#Separando a coluna preditora do target
X = df1['Verb Native']
y = df1['TIPO']

## Word Treatment and Vectorization

At this stage, it is necessary to normalize the written words, as well as to check the writing, by removing accents and punctuation.

In [111]:
# Limpeza = letras minusculas, exclusao de apostrofos, aspas e coisas do tipo
#X = hero.clean(X)

In [9]:
# Separação entre bases de treinamento e teste
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size = 0.3, random_state=100, shuffle=True)

In [10]:
#Processo de vetorização das palavras
vectorizer = TfidfVectorizer()
X_train_tfidf_vectorize = vectorizer.fit_transform(Xtrain)

## Training the classifier

In [11]:
clf = SGDClassifier()
#Fitando o classificador
clf.fit(X_train_tfidf_vectorize, ytrain) 

SGDClassifier()

In [12]:
#Vetorizando a base teste
X_test_tfidf_vectorize = vectorizer.transform(Xtest)

In [13]:
#Predizendo a base teste
predicted = clf.predict(X_test_tfidf_vectorize)

In [14]:
#Verificando a performance do modelo
print(metrics.classification_report(ytest.values, predicted))

              precision    recall  f1-score   support

    Complain       0.85      0.91      0.88      3501
     Dislike       0.77      0.64      0.70      1602

    accuracy                           0.83      5103
   macro avg       0.81      0.78      0.79      5103
weighted avg       0.82      0.83      0.82      5103



In [16]:
confusion_matrix = confusion_matrix(ytest, predicted)
print(confusion_matrix)

[[3201  300]
 [ 574 1028]]


In [17]:
predicted = pd.DataFrame(predicted)
predicted = predicted.reset_index(drop=True)
ytest = pd.DataFrame(ytest)
ytest = ytest.reset_index(drop=True)
Xtest = pd.DataFrame(Xtest)
Xtest = Xtest.reset_index(drop=True)
predicted = predicted.rename({0: 'Predito'}, axis = 1)

In [18]:
Base_Teste_Ajustada = Xtest.merge(ytest, right_index=True, left_index=True, how = "outer")

In [19]:
#Cruzamento da base textual com as estimativas de classificação textual
Base_Teste_Ajustada1 = Base_Teste_Ajustada.merge(predicted, right_index=True, left_index=True, how = "outer")

In [20]:
#Base Teste Classificada Exportada
Base_Teste_Ajustada1.to_excel("Base_Teste_Ajustada.xlsx", index=False)

## Reading from a new database

In [25]:
df = pd.read_csv('Base_Nova.csv', sep= ';', encoding="latin-1")
df.shape

(21390, 38)

In [26]:
df.dropna(subset=['Verb Native'], inplace=True)
df.shape

(13772, 38)

In [28]:
#Seleção das avaliações acima de 4 caracteres
df1 = df[df['Verb Native'].map(len) > 4]
df1 = df1.reset_index(drop=True)
df1.shape

(13738, 38)

In [29]:
#Definição da coluna a ser classificada
X = df1['Verb Native']

## Word Treatment and Vectorization

In [40]:
# Limpeza = letras minusculas, exclusao de apostrofos, aspas e demais pontuações
#X_clean = hero.clean(X)

In [30]:
#Vetorização das palavras
Base_vectorize = vectorizer.transform(X)

## Classifier Forecast

In [31]:
#Estimativas obtidas pelo modelo de classificação textual 
predicted = clf.predict(Base_vectorize)

## Exporting the New Sorted Base in xlsx Format

In [32]:
predicted = pd.DataFrame(predicted)
predicted = predicted.reset_index(drop=True)
#Renomear a coluna predict que por defaut possui o nome de 0
predicted = predicted.rename({0: 'Predito'}, axis = 1)

In [33]:
#Re-indexação da base
X = pd.DataFrame(X)
X = X.reset_index(drop=True)

In [34]:
#Cruzamento da base textual com as estimativas de classificação textual
Base_Final = df1.merge(predicted, right_index=True, left_index=True, how = "outer")

In [35]:
#Base Final Exportada
Base_Final.to_excel("Base_Escorada.xlsx", index=False)